In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download peace1019/fingerprint-dataset-for-fvc2000-db4-b

 93% 24.0M/25.9M [00:00<00:00, 81.9MB/s]
100% 25.9M/25.9M [00:00<00:00, 77.3MB/s]


In [3]:
!unzip fingerprint-dataset-for-fvc2000-db4-b.zip -d sample_data/fingerprint

Archive:  fingerprint-dataset-for-fvc2000-db4-b.zip
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/np_data/img_real.npy  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/np_data/img_train.npy  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/np_data/label_real.npy  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/np_data/label_train.npy  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00000.bmp  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00001.bmp  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00002.bmp  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00003.bmp  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00004.bmp  
  inflating: sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data/00005.bmp  
  inflating: sample_data/fingerprint/dataset_FVC

In [4]:
import matplotlib.pyplot as plt
import joblib
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
import glob
import imageio
from skimage.morphology import skeletonize
from skimage import img_as_bool, img_as_ubyte

In [5]:
def preprocess_img(img):
  img = cv2.equalizeHist(img)
  _, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  img_skeleton = skeletonize(img_bin // 255)
  img_skeleton = img_as_ubyte(img_skeleton)
  return img_skeleton

def calculate_orientation(sobelx, sobely, i, j):
  return np.rad2deg(np.arctan2(sobely[i, j], sobelx[i, j])) % 360

def extract_minutiae_features_and_orientation(skeleton, original_image):
  sobelx = cv2.Sobel(original_image, cv2.CV_64F, 1, 0, ksize=5)
  sobely = cv2.Sobel(original_image, cv2.CV_64F, 0, 1, ksize=5)

  neighbors = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
  rows, cols = skeleton.shape
  minutiae_endings = []
  minutiae_bifurcations = []
  minutiae_features = []
  x_ending = []
  y_ending = []
  x_bifurcations = []
  y_bifurcations = []
  for i in range(1, rows - 1):
    for j in range(1, cols - 1):
      if skeleton[i, j] == 255:
        cn = 0
        for k in range(8):
          p1 = skeleton[i + neighbors[k][0], j + neighbors[k][1]]
          p2 = skeleton[i + neighbors[(k + 1) % 8][0], j + neighbors[(k + 1) % 8][1]]
          cn += (p1 == 0 and p2 == 255)
        # if cn == 1 or cn == 3:
        #   orientation = calculate_orientation(sobelx, sobely, i-1, j-1)
        #   minutiae_type = 1 if cn == 1 else 3 # 1 for ending, 3 for burification
        #   minutiae_features.append({"x": j, "y": i, "type": minutiae_type, "orientation": orientation})
        if cn == 1:
          minutiae_endings.append((j, i))
          x_ending.append(j)
          y_ending.append(i)
        elif cn == 3:
          minutiae_bifurcations.append((j, i))
          x_bifurcations.append(j)
          y_bifurcations.append(i)

  minutiae_features = [np.linalg.norm(np.array(x_ending) - np.array(y_ending)),
                       np.linalg.norm(np.array(x_bifurcations) - np.array(y_bifurcations))]
  return minutiae_features

In [6]:
def construct_feature_vectors(minutiae_features_list, max_minutiae=10):
  feature_vectors = np.zeros((len(minutiae_features_list), max_minutiae * 4))

  for i, minutiae_features in enumerate(minutiae_features_list):
    for j, minutia in enumerate(minutiae_features[:max_minutiae]):
      idx = j * 4
      feature_vectors[i, idx:idx+4] = [minutia['x'], minutia['y'], minutia['type'], minutia['orientation']]

  return feature_vectors

In [7]:
# Define a function to load images from a folder
def load_images_from_folder(folder):

  # Create empty lists to store the images and labels
  images = []
  labels = []
  features = []

  # Iterate over all of the files in the folder
  for filename in os.listdir(folder):

    # Read the image into memory
    img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (160,160))
    # Check to make sure that the image is not None
    if img is not None:
      skeleton_image = preprocess_img(img)
      minutiae_feature = extract_minutiae_features_and_orientation(skeleton_image, img)
      # Add the image and label to the corresponding lists
      images.append(img)
      labels.append(int(filename.split('_')[0].split('.')[0]))
      features.append(minutiae_feature)

  # Return the images and labels
  return images, labels, features

# Load the training and test datasets
x_train, y_train, feature_train = load_images_from_folder('/content/sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/train_data')
x_test, y_test, feature_test = load_images_from_folder('/content/sample_data/fingerprint/dataset_FVC2000_DB4_B/dataset/real_data')

# Print the number of training images and the shape of the first training image
print("Number of training images:", len(x_train))
print("Shape of the first training image:", x_train[0].shape)

Number of training images: 800
Shape of the first training image: (160, 160)


In [8]:
feature_train_list = []
for i in range(800):
  print(y_train[i], feature_train[i])


# feature vector Euclidean distance ending points and bifurcation points

1 [1133.2409276054232, 852.5702317111477]
3 [605.0140494236476, 827.8230487247863]
0 [1082.0383542185555, 1308.9984721152275]
0 [873.2989178969592, 1041.6314127367702]
4 [474.5640104348411, 1736.0947554785137]
8 [1019.5631417425799, 1455.2058273660123]
0 [1001.6042132499243, 1352.3771663260216]
9 [884.2030309832692, 951.4215679707918]
8 [954.385666279623, 1284.5018489671395]
6 [603.5668977006609, 1412.1426981718243]
0 [1198.844026552245, 1010.3217309352501]
1 [856.4817569569127, 1120.2602376233835]
8 [648.1134160006256, 1917.7265185630615]
4 [960.4118908051898, 1554.9379408838154]
0 [1206.582777931129, 1102.278549187999]
1 [817.7670817537228, 1035.5650631418578]
1 [767.1805524125334, 1358.4428585700614]
8 [1155.3947377411757, 1159.9310324325322]
0 [1173.0664942789901, 1138.045693283007]
7 [644.006211150172, 1510.1658849278776]
9 [1057.7372074385962, 1251.9628588740163]
9 [1011.1122588516074, 862.651146176715]
3 [557.6809123504229, 842.7105078257895]
8 [693.4601646814328, 1629.104048242

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam


In [14]:
def create_feature_model(input_shape):
  # Build the feature extraction model
  input = Input(shape=input_shape)
  x = Conv2D(32, (3, 3), activation='relu')(input)
  x = Flatten()(x)
  x = Dense(128, activation='relu')(x)
  return Model(input, x)

def distance(vectors):
  vector1, vector2 = vectors
  return tf.sqrt(tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True))

def create_network(input_shape):
  feature_model = create_feature_model(input_shape)
  # Two input layers for the two images
  input_a = Input(shape=input_shape)
  input_b = Input(shape=input_shape)
  # Process each input in the shared feature model
  features_a = feature_model(input_a)
  features_b = feature_model(input_b)

  # Distance layer
  dist = Lambda(distance)([features_a, features_b])
  return Model(inputs=[input_a, input_b], outputs=dist)

In [16]:
input_shape = (160, 160, 1)
model = create_network(input_shape)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy')
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 model_5 (Functional)        (None, 128)                  1022529   ['input_11[0][0]',            
                                                          92         'input_12[0][0]']            
                                                                                                  
 lambda_2 (Lambda)           (None, 1)                    0         ['model_5[0][0]',       